In [34]:
import numpy as np
import pandas as pd
import requests
import os
import json
from bs4 import BeautifulSoup 
import collections
collections.Callable = collections.abc.Callable

In [5]:
propublica_token = os.environ['propublica_token']

useragent_url = 'https://httpbin.org/user-agent'
r = requests.get(useragent_url)
useragent = json.loads(r.text)['user-agent']
useragent

'python-requests/2.28.1'

In [6]:
headers = {'X-API-Key': propublica_token,
          'User-Agent':useragent,
          'From': 'jkropko@virginia.edu'}

# Goal: Get the bills sponsored by Bob Good
## Step 1: Get Bob Good's ID number

In [9]:
root = "https://api.propublica.org"
congress = '117'
chamber = 'house'
endpoint = "/congress/v1/{congress}/{chamber}/members.json".format(congress = congress,
                                                                  chamber = chamber)
r = requests.get(root + endpoint,
                headers = headers)
myjson = json.loads(r.text)
membersdf = pd.json_normalize(myjson, record_path = ['results', 'members'])
membersdf.head(3).T

,0,1,2
id,A000370,A000055,A000371
title,Representative,Representative,Representative
short_title,Rep.,Rep.,Rep.
api_uri,https://api.propublica.org/congress/v1/members...,https://api.propublica.org/congress/v1/members...,https://api.propublica.org/congress/v1/members...
first_name,Alma,Robert,Pete
middle_name,None,B.,None
last_name,Adams,Aderholt,Aguilar
suffix,None,None,None
date_of_birth,1946-05-27,1965-07-22,1979-06-19
gender,F,M,M


In [11]:
bobgood = membersdf.query("last_name == 'Good'")

In [16]:
bobgoodid = bobgood.reset_index()['id'][0]

## Step 2: Use BG's ID to query the bills API 

In [19]:
endpoint = '/congress/v1/members/{memberid}/bills/{billtype}.json'.format(memberid = bobgoodid,
                                                                         billtype = 'introduced')
r = requests.get(root + endpoint,
                headers = headers)
myjson = json.loads(r.text)
bgbills1 = pd.json_normalize(myjson, record_path = ['results', 'bills'])

In [20]:
r = requests.get(root + endpoint,
                headers = headers,
                params = {'offset': 20})
myjson = json.loads(r.text)
bgbills2 = pd.json_normalize(myjson, record_path = ['results', 'bills'])

In [22]:
bgbills = pd.concat([bgbills1, bgbills2], ignore_index=True)
bgbills.head(3).T

,0,1,2
congress,117,117,117
bill_id,hr8935-117,hr8767-117,hres1297-117
bill_type,hr,hr,hres
number,H.R.8935,H.R.8767,H.RES.1297
bill_uri,https://api.propublica.org/congress/v1/117/bil...,https://api.propublica.org/congress/v1/117/bil...,https://api.propublica.org/congress/v1/117/bil...
title,To amend the Labor-Management Reporting and Di...,To establish a private right of action for par...,"Designating the week beginning November 7, 202..."
short_title,To amend the Labor-Management Reporting and Di...,Empowering Parents Act,"Designating the week beginning November 7, 202..."
sponsor_title,Rep.,Rep.,Rep.
sponsor_id,G000595,G000595,G000595
sponsor_name,Robert Good,Robert Good,Robert Good


In [46]:
urltoscrape = bgbills['congressdotgov_url'][11] + '/text'

In [47]:
r = requests.get(urltoscrape,
                headers = {'User-Agent': useragent,
                          'From': 'jkropko@virginia.edu'})
myhtml = BeautifulSoup(r.text, 'html.parser')

In [48]:
myhtml.find_all('p', "lbexIndent", style="text-align:left")[0].text

' Be it enacted by the Senate and House of Representatives of the\n\t\tUnited States of America in Congress assembled,  SECTION 1.  Short title.This Act may be cited as the “Championing our Parents Act of 2021”. SEC. 2.  Prohibition on use of Federal funds.No Federal funds may be expended to implement the memorandum from the Attorney General entitled “Partnership among Federal, State, Local, Tribal, and Territorial Law Enforcement to address threats against School Administrators, Board Members, Teachers, and Staff” dated October 4, 2021. '